In [ ]:
import sys
  
# append the path of the parent directory
sys.path.append("../")

from src import imageUtils as img
import tesserocr
from PIL import Image
import re

In [ ]:
#### CONSTANTS #######
pathToVideo = r'C:\Users\hkuser\Videos\2023-03-17 09-25-05.mkv'
outputPath = 'output'

In [ ]:
frames = img.getFramesFromVideo(pathToVideo=pathToVideo, outputPath=outputPath, shouldOutputToFile=True)

In [ ]:
height, width

In [ ]:
np.array([[1,1],[1,1]])

In [ ]:
height,width = 1080,1920

from dataclasses import dataclass,field
from datetime import datetime, timedelta
from PIL import Image, ImageDraw, ImageFont

import imagehash
import os

hash = imagehash.average_hash(im)
otherhash = imagehash.average_hash(im2)

@dataclass
class Item:
    box:tuple #left upper right lower
    name:str
    sides:list
    numbersOnly:bool
    tesserocrOptions:dict=field(default_factory=lambda: dict())
    maximumDistanceForStoredImages:int=field(default_factory=lambda: 0)

UNKNOWN_STR = '<<UNKNOWN>>'
    




BOUNDING_BOXES = {
    'IN_GAME' :
    {
        'Time': Item((875,0,1040,65), 'Time', ['none'], False),
        'TeamName': Item((185,85,400,125),'TeamName', ['left','right'], False),
        'Score': Item((700,20,825,82),'Score', ['left', 'right'], True, tesserocrOptions={'PageSegMode': tesserocr.PSM.SINGLE_WORD})    
    },
    'GOAL_SCORED_LEFT_HAND_SIDE' :
    {
        'Time': Item((250,850,380,900),'Time', ['none'], False),
        'ScoreLeft': Item((1100,735,1330,900), 'ScoreLeft', ['none'], True),
        'ScoreRight': Item((1450,735,1680,900),'ScoreRight', ['none'], True),
        'TeamName':  Item((490,850,900,900),'TeamName', ['none'], False),
        'GoalText': Item((250,925,1000,1000),'ScoreText', ['none'], False)
    },
    'IN_GAME_FINAL_RESULT_LEFT_HAND_SIDE' :
    {
        'ScoreLeft': Item((1100,735,1330,900), 'ScoreLeft', ['none'], True),
        'ScoreRight': Item((1450,735,1680,900),'ScoreRight', ['none'], True),
        'TeamName':  Item((330,850,700,900),'TeamName', ['none'], False),
        'GoalText': Item((250,925,1000,1000),'ScoreText', ['none'], False)
    }, 
}

IMAGE_STORE_FOLDER_PATH = "../storedImages"


STORED_IMAGES = ImageStore(IMAGE_STORE_FOLDER_PATH)

def flipItem(item:Item) -> Item:
    return Item((1920 - item.box[2], item.box[1], 1920-item.box[0], item.box[3]), item.name, item.sides, item.numbersOnly)

BOUNDING_BOXES['GOAL_SCORED_RIGHT_HAND_SIDE'] = {k:flipItem(v) for k,v in BOUNDING_BOXES['GOAL_SCORED_LEFT_HAND_SIDE'].items()}
BOUNDING_BOXES['IN_GAME_FINAL_RESULT_RIGHT_HAND_SIDE'] = {k:flipItem(v) for k,v in BOUNDING_BOXES['IN_GAME_FINAL_RESULT_LEFT_HAND_SIDE'].items()}

def drawBoxes(im:Image, moment:str) -> Image:
    draw = ImageDraw.Draw(im)
    font = ImageFont.truetype("arial")
    momentConfig = BOUNDING_BOXES[moment]
    for key,item in momentConfig.items():
        for side in item.sides:
            draw.rectangle(getSubImageBox(item, side), outline='red')
            text = f'{key}/{side}'
           # draw.text((20, 20), text, font=font)
    return im


def parseToActualValue(key:str, val:str):
    if key == 'Time':
        return parseTimeDuration(val)
    elif key.startswith('Score'): 
        if val == '' or int(val) is None:
            raise ValueError(f'Cannot parse int from the provided value `{val}` for the key {key}')
        return int(val)
    return val

def parseTimeDuration(timeStr:str) -> timedelta:
    minutes, seconds = None, None
    if ':' in timeStr:
        minutes,seconds = timeStr.split(':')[0], timeStr.split(':')[1]
    elif len(timeStr) == 4:
        minutes = int(timeStr[0])*10 + int(timeStr[1])
        seconds = int(timeStr[2])*10 + int(timeStr[3])
    else:
        raise ValueError(f'`{timeStr}` could not be parsed into time duration.')
    return timedelta(minutes=int(minutes), seconds=int(seconds))

def imageTransformationForScore(im:Image) -> Image:
    tmp = im
    tmp = cv2.cvtColor(np.array(tmp), cv2.COLOR_BGR2GRAY)
    (th, newimg) = cv2.threshold(tmp, 215, 255, cv2.THRESH_BINARY)
    #lower = np.array([0,0,196])
    #upper = np.array([179,39,255])
    #mask = cv2.inRange(tmp, lower, upper)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
    opening = cv2.morphologyEx(newimg, cv2.MORPH_OPEN, kernel)
    kernel = np.ones((3,3),np.uint8)
    erosion = cv2.erode(opening,kernel,iterations = 2)
    return Image.fromarray(255-erosion)

def imageTransformation(im:Image, type:str) -> list:
    res = []
    if type.startswith('Score'):
        res.append(imageTransformationForScore(im))
    elif type.startswith('Time'):
        kernel = np.ones((3,3),np.uint8)
        erosion = cv2.erode(np.array(im),kernel,iterations = 1)
        im1 = Image.fromarray(erosion)
        res.append(im1)
    elif type.startswith('TeamName'):
        tmp = cv2.cvtColor(np.array(im), cv2.COLOR_BGR2GRAY)
        (th, newimg) = cv2.threshold(tmp, 50, 255, cv2.THRESH_BINARY)
        res.append(Image.fromarray(newimg))
    else:
        print('No transformation found')
        res.append(im)
    return res

def getSubImageBox(item:Item, side:str) -> tuple:
    validSides = {'left', 'right', 'none'}
    if side not in validSides:
        raise ValueError(f'side must be {validSides}')
    
    box = item.box
    if side == 'right':
        box = (width - box[2], box[1], width - box[0], box[3])
    return box

def parseImage(im:Image, moment:str, item:Item, debug:bool=False) -> str:
    value, dist = STORED_IMAGES.lookupImageValue(im, moment, item.name)
    if value is not None:
        if debug:
            print(f'Found item in STORED IMAGES with distance of `{dist}` and value of `{value}`')
        return value
    with tesserocr.PyTessBaseAPI() as api:
        api.SetPageSegMode(tesserocr.PSM.SINGLE_LINE)
        for option,value in tesserocrOptions.items():
            api.SetVariable(option, value)
        if numbersOnly:
            api.SetVariable("tessedit_char_whitelist", "0123456789")
        api.SetImage(im)
        res = api.GetUTF8Text()
        return re.sub(r'\n', '', res), api.AllWordConfidences()


def parseFromImage(im:Image, item:Item, side:str, debug:bool=False):
    try:
        imgs = imageTransformation(im, f'{key}/{side}')
    except Exception as err:
        print(f'Exception encountered when trying to transform image for {key} side {side}')
        raise err
    for imgVal in imgs:
        parsedVal, confidences = parseImage(imgVal, item.numbersOnly)
        if debug:
            display(f'ParsedValue: `{parsedVal}`')
            display(f'Confidences: {confidences}')
            display('Image parsed:')
            display(imgVal)
        try:
            parsedVal = parseToActualValue(key,parsedVal)
        except Exception as err:
            if debug:
                print(f'Exception when parsing {key}. Text=`{parsedVal}`')
            raise err
        return parsedVal, confidences
    return None, [0]


def getAllItemsFromImage(im:Image, debug:bool=False) -> dict:
    res = {}
    skipMoment = False

    for moment,boxes in BOUNDING_BOXES.items():
        skipMoment=False
        if debug:
            display("Extracting the following areas from the image:")
            display(drawBoxes(im.copy(), moment))
        firstValue = True
        for key,item in boxes.items():
            for side in item.sides:
                imgs = None
                try:
                    box = getSubImageBox(item, side)
                    cropped = im.crop(box)
                    if debug:
                        print(f'Outputting cropped image for {key} side {side}, box={box}')
                        display(cropped)
                    imgs = imageTransformation(cropped, f'{key}/{side}')
                except Exception as err:
                    print(f'Exception encountered when trying to transform image for {key} side {side}')
                    raise err
                for imgVal in imgs:
                    parsedVal, confidences = parseImage(imgVal, item.numbersOnly)
                    if debug:
                        display(f'ParsedValue: `{parsedVal}`')
                        display(f'Confidences: {confidences}')
                        display('Image parsed:')
                        display(imgVal)
                    try:
                        parsedVal = parseToActualValue(key,parsedVal)
                    except Exception as err:
                        if debug:
                            print(f'Exception when parsing {key}. Text=`{parsedVal}`')
                        if firstValue:
                            print(f'Was not {moment} because {key} could not be parsed. Text=`{parsedVal}`')
                            print(f'{err}')
                            skipMoment=True
                            continue
                    skipMoment=False
                    res[f'{key}/{side}'] = {'image': imgVal, 'value': parsedVal, 'confidences': confidences}
                    break
                if skipMoment:
                    break
            if skipMoment:
                break
            firstValue = False
        if not skipMoment:
            parsedObjs = {}
            parsedObjs['Type'] = moment
            parsedObjs['Values'] = res
            return parsedObjs      
    return {}

import numpy as np


def getResultsPerFrame(frames:list) -> list:
    return [getAllItemsFromImage(Image.fromarray(im)) for im in frames]

In [ ]:
import cv2
import numpy as np

def nothing(x):
    pass

# Load image
image = frames[166]

# Create a window
cv2.namedWindow('image')

# Create trackbars for color change
# Hue is from 0-179 for Opencv
cv2.createTrackbar('HMin', 'image', 0, 179, nothing)
cv2.createTrackbar('SMin', 'image', 0, 255, nothing)
cv2.createTrackbar('VMin', 'image', 0, 255, nothing)
cv2.createTrackbar('HMax', 'image', 0, 179, nothing)
cv2.createTrackbar('SMax', 'image', 0, 255, nothing)
cv2.createTrackbar('VMax', 'image', 0, 255, nothing)

# Set default value for Max HSV trackbars
cv2.setTrackbarPos('HMax', 'image', 179)
cv2.setTrackbarPos('SMax', 'image', 255)
cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize HSV min/max values
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

while(1):
    # Get current positions of all trackbars
    hMin = cv2.getTrackbarPos('HMin', 'image')
    sMin = cv2.getTrackbarPos('SMin', 'image')
    vMin = cv2.getTrackbarPos('VMin', 'image')
    hMax = cv2.getTrackbarPos('HMax', 'image')
    sMax = cv2.getTrackbarPos('SMax', 'image')
    vMax = cv2.getTrackbarPos('VMax', 'image')

    # Set minimum and maximum HSV values to display
    lower = np.array([hMin, sMin, vMin])
    upper = np.array([hMax, sMax, vMax])

    # Convert to HSV format and color threshold
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(image, image, mask=mask)

    # Print if there is a change in HSV value
    if((phMin != hMin) | (psMin != sMin) | (pvMin != vMin) | (phMax != hMax) | (psMax != sMax) | (pvMax != vMax) ):
        print("(hMin = %d , sMin = %d, vMin = %d), (hMax = %d , sMax = %d, vMax = %d)" % (hMin , sMin , vMin, hMax, sMax , vMax))
        phMin = hMin
        psMin = sMin
        pvMin = vMin
        phMax = hMax
        psMax = sMax
        pvMax = vMax

    # Display result image
    cv2.imshow('image', result)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [ ]:


tmp = im.crop(getSubImageBox(BOUNDING_BOXES['IN_GAME']['Score'], 'left'))
tmp = imageTransformationForScore(tmp)
display(tmp)
display(f'Parsed value: {parseImage(tmp, True)}')

tmp = im.crop(getSubImageBox(BOUNDING_BOXES['IN_GAME']['Score'], 'right'))
tmp = imageTransformationForScore(tmp)
display(tmp)
display(f'Parsed value: {parseImage(tmp, True)}')

In [ ]:
np.array(tmp)

In [ ]:
#ogimg = frames[1] #cv2.cvtColor(frames[1], cv2.COLOR_BGR2GRAY)
#ogimg= cv2.adaptiveThreshold(cv2.GaussianBlur( (5,5),2),255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                         cv2.THRESH_BINARY, 155, 40)

#ogimg = cv2.cvtColor(frames[1], cv2.COLOR_BGR2GRAY)

# Otsu's thresholding

ogimg = frames[166]

im = Image.fromarray(ogimg)
items = getAllItemsFromImage(im, debug=True)
if 'Values' not in items:
    print('Could not parse this frame.')
    print(items)
else:
    for key,val in items['Values'].items():
        display(key)
        display(val['image'])
        display(val['value'])

In [ ]:
''==''

In [ ]:
items

In [ ]:

cropped = im.crop(getSubImageBox(BOUNDING_BOXES["IN_GAME"]["Time"], 'none'))
display(cropped)

kernel = np.ones((3,3),np.uint8)
erosion = cv2.erode(np.array(cropped),kernel,iterations = 1)
output = Image.fromarray(erosion)
display(output)
display(parseImage(output, False))

In [ ]:
res = getResultsPerFrame(frames)
res

In [ ]:
for i in range(len(res)):
    if len(res[i]) == 0:
        display(f'Frame {i}')
        display(Image.fromarray(frames[i]))


In [ ]:
print(tesserocr.tesseract_version())  # print tesseract-ocr version
print(tesserocr.get_languages())  # prints tessdata path and list of available languages

In [ ]:
cnt = 0
skipXFrames = 20
while cnt < 25:
    while i < len(res):
        entry = res[i]
        if len(entry) == 0:
            i+=1
            continue
        if entry['Type'] == 'IN_GAME':
            for side in ['left', 'right']:
                cropped = Image.fromarray(frames[i]).crop(getSubImageBox(BOUNDING_BOXES['IN_GAME']['Score'], side))
                cv2.imwrite(f'../tests/qualityData/score/{side}/score_{side}_{cnt}.jpg', np.array(cropped))
            cnt += 1
            i += skipXFrames
        if cnt >= 25:
            break
        i += 1


In [ ]:
import os
import glob

class TestRunner(object):
    QUALITY_TESTS = os.path.join('..', 'tests', 'qualityData')

    def __init__(self):
        pass

    def _getTestNumber(self,fileName:str) -> int:
        return int(fileName.split('_')[-1].split('.')[0])

    def _getBaseFolderPath(self, moment:str, testCase:str, side:str) -> str:
        return os.path.join(TestRunner.QUALITY_TESTS, testCase.lower(), side)
    
    def _getCorrectResults(self, moment:str, testCase:str, side:str) -> list:
        with open(os.path.join(self._getBaseFolderPath(moment, testCase, side), 'labels.txt')) as file:
            lines = {idx:line.rstrip() for idx,line in enumerate(file)}
        return lines

    def _convertToCorrectType(self, value:str, testCase:str):
        if testCase.lower().startswith('score'):
            return int(value)
        return value

    def getTestResults(self, moment:str, testCase:str, side:str, considerBelowXConfidenceAsFailing:float=70, debug:bool=False):
        imgs = {}
        totalTestCases = 0
        correct = 0
        confidences = []
        answers = self._getCorrectResults(moment, testCase, side)
        for filename in glob.glob(os.path.join(self._getBaseFolderPath(moment, testCase, side),'*.jpg')):
            im=Image.open(filename)
            testNo = self._getTestNumber(filename)
            try:
                parsedVal, confidences = parseFromImage(im, BOUNDING_BOXES[moment][testCase], side, debug=debug)
            except:
                parsedVal, confidences = '', [0]
            totalTestCases += 1
            confidence = np.min(confidences)
            if parsedVal == self._convertToCorrectType(answers[testNo], testCase) and confidence >= considerBelowXConfidenceAsFailing:
                correct += 1
            elif debug:
                display(f'Failed to properly parse the following image / had low confidence | testNo={testNo}')
                display(f'Correct value `{answers[testNo]}`, parsed value `{parsedVal}`, passing confidence was {considerBelowXConfidenceAsFailing}, actual confidence was {confidence}')
                display(im)
            confidences.append(confidence)
        print(f'Final results for {testCase} | {side} ')
        print(f'% correct = {100.0*correct/totalTestCases}% ({correct} out of {totalTestCases})')
        print(f'Confidence (p0, p25, p50, p75): {np.percentile(confidences, [0, 25, 50, 75])}')
            

In [ ]:
tests = TestRunner()
tests.getTestResults('IN_GAME', 'Score', 'left', debug=True)

In [ ]:
def imageTransformationForScore(im:Image) -> Image:
    tmp = im
    tmp = cv2.cvtColor(np.array(tmp), cv2.COLOR_BGR2GRAY)
    (th, newimg) = cv2.threshold(tmp, 215, 255, cv2.THRESH_BINARY)
    #lower = np.array([0,0,196])
    #upper = np.array([179,39,255])
    #mask = cv2.inRange(tmp, lower, upper)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
    opening = cv2.morphologyEx(newimg, cv2.MORPH_OPEN, kernel)
    kernel = np.ones((1,1),np.uint8)
    erosion = cv2.erode(opening,kernel,iterations = 5)
    return Image.fromarray(255-erosion)

def parseImage(im:Image, numbersOnly:bool=False, tesserocrOptions:dict={}) -> str:
    with tesserocr.PyTessBaseAPI() as api:
        api.SetPageSegMode(tesserocr.PSM.SINGLE_LINE)
        if 'PageSegMode' in tesserocrOptions:
            api.SetPageSegMode(tesserocrOptions['PageSegMode'])
        if numbersOnly:
            api.SetVariable("tessedit_char_whitelist", "0123456789")
        api.SetImage(im)
        res = api.GetUTF8Text()
        return re.sub(r'\n', '', res), api.AllWordConfidences()


def parseFromImage(im:Image, item:Item, side:str, debug:bool=False):
    try:
        imgs = imageTransformation(im, f'{key}/{side}')
    except Exception as err:
        print(f'Exception encountered when trying to transform image for {key} side {side}')
        raise err
    for imgVal in imgs:
        parsedVal, confidences = parseImage(imgVal, item.numbersOnly)
        if debug:
            display(f'ParsedValue: `{parsedVal}`')
            display(f'Confidences: {confidences}')
            display('Image parsed:')
            display(imgVal)
        try:
            parsedVal = parseToActualValue(key,parsedVal)
        except Exception as err:
            if debug:
                print(f'Exception when parsing {key}. Text=`{parsedVal}`')
            raise err
        return parsedVal, confidences
    return None, [0]



In [ ]:
[e.value for e in tesserocr.PSM]

In [ ]:
im.resize((int(im.size[0]*1.5), int(im.size[1])), resample=Image.NEAREST)

In [ ]:
def imageTransformationForScore(im:Image) -> Image:
    tmp = im
    tmp = cv2.cvtColor(np.array(tmp), cv2.COLOR_BGR2GRAY)
    (th, newimg) = cv2.threshold(tmp, 215, 255, cv2.THRESH_BINARY)
    #lower = np.array([0,0,196])
    #upper = np.array([179,39,255])
    #mask = cv2.inRange(tmp, lower, upper)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
    opening = cv2.morphologyEx(newimg, cv2.MORPH_OPEN, kernel)
    kernel = np.ones((1,1),np.uint8)
    erosion = cv2.erode(opening,kernel,iterations = 5)
    return Image.fromarray(255-erosion)

im = Image.open(r"C:\wd\MSBL-Parser\tests\qualityData\in_game\score\left\score_left_10.jpg")
im = imageTransformation(im, 'Score')[0]
im.save("2.jpg")
display(im)

for val in [tesserocr.PSM.OSD_ONLY, tesserocr.PSM.AUTO_OSD, tesserocr.PSM.AUTO_ONLY, tesserocr.PSM.AUTO, tesserocr.PSM.SINGLE_COLUMN, tesserocr.PSM.SINGLE_BLOCK_VERT_TEXT, tesserocr.PSM.SINGLE_BLOCK, tesserocr.PSM.SINGLE_LINE, tesserocr.PSM.SINGLE_WORD, tesserocr.PSM.CIRCLE_WORD, tesserocr.PSM.SINGLE_CHAR, tesserocr.PSM.RAW_LINE]:
    display(f'PSM option: {val}')
    display(parseImage(im,  numbersOnly=True, tesserocrOptions={'psm': val, 'oem':1}))

In [ ]:
im = Image.open(r"C:\wd\MSBL-Parser\tests\qualityData\in_game\score\left\score_left_10.jpg")
im = imageTransformation(im, 'Score')[0]

#im2 = Image.open(r"2.jpg")
im2 = Image.open(r"C:\wd\MSBL-Parser\tests\qualityData\in_game\score\right\score_right_10.jpg")

import imagehash

hash = imagehash.average_hash(im)
otherhash = imagehash.average_hash(im2)

print(hash - otherhash)